# Repairs of Python.NET 3 breaking changes to low-level examples

## Load a runtime before calling `import clr`

In order to access .NET assemblies (`.dll` files), one must load an available runtime before executing the
`import clr` statement. (If one calls `import clr` before specifying a runtime, Python.NET will load a default
runtime which may **not** be compatible with the installed Orchid assemblies.

To make this easier, when we `import` the `orchid` package, the Orchid Python API will load the runtime
corresponding to the configured Orchid installation.

In [ ]:
import orchid

In [ ]:
# Remember that importing `orchid` results in a call to `pythonnet.load()`
# to specify the targeted runtime:
#
# 'netfx': .NET Framework
# 'coreclr': .NET Core
# 'mono': Mono

import clr

In [ ]:
from System import (
    ArgumentException,
    Array,
    DateTime,
    TimeSpan,
    Int32,
    ValueType,
)

## Fewer implicit conversions between Python values and .NET values

### Adding attributes with integer values requires an explicit conversion

(This issue occured in **both** internal testing and low-level script testing and so is duplicated.)

During testing, we discovered an issue setting an attribute with type, `Int32`, using a Python `int` 
value of 7. The run-time reported that the types, `Int32` and `PyInt` were incompatible. 

This scenario requires significant set up.

In [ ]:
# Find the well named 'Demo_1H'
bakken = orchid.load_project('c:/src/Orchid.IntegrationTestData/frankNstein_Bakken_UTM13_FEET.ifrac')
candidate_wells = list(bakken.wells().find_by_name('Demo_1H'))
assert len(candidate_wells) == 1
demo_1h = candidate_wells[0]

In [ ]:
# Create an attribute with name, 'My New Attribute', and type, `System.Int32`
# noinspection PyUnresolvedReferences,PyPackageRequirements
from Orchid.FractureDiagnostics.Factories.Implementations import Attribute

attribute_to_add_type = Int32
attribute_to_add = Attribute[attribute_to_add_type].Create('My New Attribute')

In [ ]:
# Add newly created attribute to well, 'Demo_1H'
with orchid.dot_net_disposable.disposable(demo_1h.dom_object.ToMutable()) as mutable_well:
    mutable_well.AddStageAttribute(attribute_to_add)

In [ ]:
# Find stage number 7 in well, 'Demo_1H'
maybe_stage = demo_1h.stages().find_by_display_stage_number(7)
assert maybe_stage is not None
stage_7 = maybe_stage

In [ ]:
# Add attribute with value, 17, to stage 7, with Python `int` type.
with (orchid.dot_net_disposable.disposable(stage_7.dom_object.ToMutable())) as mutable_stage:
    # This action will fail because the attribute type is `System.Int32`
    # and `pythonnet-3.0.0.post1` **does not** implicitly equate these two types.
    try:
        mutable_stage.SetAttribute(attribute_to_add, int)
    except ArgumentException as ae:
        print(f'ArgumentException: {ae}')


In [ ]:
# Add attribute to stage 7 with a value of 17 **explicitly** converted to an `Int32`
with (orchid.dot_net_disposable.disposable(stage_7.dom_object.ToMutable())) as mutable_stage:
    mutable_stage.SetAttribute(attribute_to_add, attribute_to_add_type(7))

In [ ]:
# Verify added attribute value
ignored_object = object()
is_attribute_present, actual_attribute_value = stage_7.dom_object.TryGetAttributeValue(attribute_to_add,
                                                                                       ignored_object)
assert is_attribute_present
assert type(actual_attribute_value) == int
assert actual_attribute_value == 7

### Leakoff.ControlPoints and no ValueType() ctor error

This issue seems similar to the internal test issue in which the .NET `TimeSpan` class did not have a default
constructor, but Python.NET 2.5.2 accepted the expression, `TimeSpan()`, and appeared to "do the right thing."

In this situation, our low-level example code contained the expression:

```
Leakoff.ControlPoint(DateTime=some_time, Pressure=some_pressure)
```

Although Python.NET 2.5.2 seemed to "do the right thing" with this expression, executing this same expression using
Python.NET 3 encounters an exception with a somewhat obscure (in retrospect) error message.

In [ ]:
# noinspection PyUnresolvedReferences
from Orchid.FractureDiagnostics import Leakoff

clr.AddReference('System.Collections')
# noinspection PyUnresolvedReferences
from System.Collections.Generic import List
# noinspection PyUnresolvedReferences
import UnitsNet

In [ ]:
end = DateTime.UtcNow
start = end.Subtract(TimeSpan.FromMinutes(10))
error_control_points = List[Leakoff.ControlPoint]()

In [ ]:
try:
    error_control_points.Add(
        Leakoff.ControlPoint(
            Date=start,
            Pressure=UnitsNet.Pressure(95.64, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)
        ),
        Leakoff.ControlPoint(
            Date=end,
            Pressure=UnitsNet.Pressure(104.8, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)
        )
    )
except TypeError as te:
    print(f'TypeError: {te}')

The Orchid team ran some experiments in C#. We observed:

- Code that created a `List<Leakoff.ControlPoint>()` and then using
  `List.Add()` ran without any errors.
- Code that tried to invoke the "constructor",
  `Leakoff.ControlPoint(DateTime, Pressure)` failed to compile.

These observations led us to the hypothesis that `pythonnet-2.5.2` performed additional work to convert the Python 
expression `Leakoff.ControlPoint(DateTime.UtcNow, Pressure.FromPoundsForcePerSquareInt(100))` into the equivalent 
C# code:

```
var controlPoint = new Leakoff.ControlPoint();
controlPoint.Date = DateTime.UtcNow;
controlPoint.Pressure = Pressure.FromPoundsForcePerSquareInch(100);
```

The following cell tests that hypothesis.

In [ ]:
python_control_points = [Leakoff.ControlPoint(), Leakoff.ControlPoint()]
python_control_points[0].DateTime = start
python_control_points[0].Pressure = UnitsNet.Pressure(95.64, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)
python_control_points[1].DateTime = start
python_control_points[1].Pressure = UnitsNet.Pressure(104.8, UnitsNet.Units.PressureUnit.PoundForcePerSquareInch)

The following code is similar.

In [ ]:
working_control_points = List[Leakoff.ControlPoint]()
working_control_points.Add(python_control_points[0])
working_control_points.Add(python_control_points[1])

## .NET Collections and arrays no longer automatically converted to Python collections (like `list`)

Python.NET 2.5.2 implicitly converted .NET Collections into Python collections. These implicit conversions occurred
both for types like `List[Int32]` and for other similar third-party types like `DynamicData.Items`. Consequently, 
using `len` on some .NET Collection value acts as expected but unexpectedly for other value.

In [ ]:
a_net_list = List[Int32]()
a_net_list.Add(1)
a_net_list.Add(2)
a_net_list.Add(3)
try:
    len(a_net_list) == 3
    print('Eureka!')
except TypeError as te:
    print(f'TypeError: {te}')

In [ ]:
type(a_net_list)

In [ ]:
import collections.abc

isinstance(a_net_list, collections.abc.Collection)

[This comment](https://github.com/pythonnet/pythonnet/issues/1153#issuecomment-926143701) indicates that the 
`pythonnet-2.5.2` implicit conversions from .NET `List` to a Python `list` is no longer supported. 

In [ ]:
isinstance(a_net_list, list)

### Error raised when calling `len` with instance of .NET DynamicData

In [ ]:
net_observation_sets_items = bakken.dom_object.ObservationSets.Items
type(net_observation_sets_items)

In [ ]:
isinstance(net_observation_sets_items, collections.abc.Collection)

In [ ]:
isinstance(net_observation_sets_items, collections.abc.Container)

In [ ]:
isinstance(net_observation_sets_items, collections.abc.Iterator)

In [ ]:
isinstance(net_observation_sets_items, collections.abc.Sequence)

In [ ]:
isinstance(net_observation_sets_items, collections.abc.Iterable)

In [ ]:
observation_sets = []
for observation_set_item in bakken.dom_object.ObservationSets.Items:
    observation_sets.append(observation_set_item)
observation_sets

In [ ]:
len(observation_sets)

In [ ]:
alt_observation_sets = [i for i in bakken.dom_object.ObservationSets.Items]
alt_observation_sets